In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


df = pd.read_csv('ratings.csv')
df = df[['userId', 'movieId', 'rating']]

(Encoding)
user_enc = LabelEncoder()
df['user'] = user_enc.fit_transform(df['userId'].values)
movie_enc = LabelEncoder()
df['movie'] = movie_enc.fit_transform(df['movieId'].values)

n_users = df['user'].nunique()
n_movies = df['movie'].nunique()


train, test = train_test_split(df, test_size=0.2, random_state=42)

(Embedding Model)
class RecommenderNet(nn.Module):
    def __init__(self, n_users, n_movies, emb_size=50):
        super(RecommenderNet, self).__init__()
        self.user_emb = nn.Embedding(n_users, emb_size)
        self.movie_emb = nn.Embedding(n_movies, emb_size)
        
        self.fc = nn.Linear(emb_size, 1)
        
    def forward(self, user, movie):
        user_vec = self.user_emb(user)
        movie_vec = self.movie_emb(movie)
        (Dot Product) 
        dot = (user_vec * movie_vec).sum(1)
        return dot

model = RecommenderNet(n_users, n_movies)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)



users = torch.LongTensor(train.user.values)
movies = torch.LongTensor(train.movie.values)
ratings = torch.FloatTensor(train.rating.values)


for epoch in range(5): 
    model.train()
    optimizer.zero_grad()
    preds = model(users[:10000], movies[:10000]) 
    loss = criterion(preds, ratings[:10000])
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")

print("DONE!!")